In [10]:
import torch as th
import numpy as np


class Euclidean:
    def __init__(self, max_norm=None, K=None, **kwargs):
        self.max_norm = max_norm
        self.K = K
        if K is not None:
            self.inner_radius = 2 * self.K / (1 + np.sqrt(1 + 4 * self.K * self.K))

    def init_weights(self, w, scale=1e-4):
        w.weight.data.uniform_(-scale, scale)

    def distance(self, u, v):
        return ((u - v).pow(2)).pow(1/2).sum(dim=-1)

from torch.nn import Module
from torch.nn import Embedding
import torch.nn.functional as fun
from torch import zeros


class Model(Module):
    def __init__(self, manifold, n, dim, sparse=False):
        super().__init__()
        self.manifold = manifold
        self.n = n
        self.dim = dim
        self.model = Embedding(n, dim, sparse=sparse)

    def forward(self, inputs):  # z inputs zrobi preds - z macierzy 10x52 zrobi sie 10x52xdim
        e = self.model(inputs)  # macierz 10x52xdim, to bedzie e to preds z pliku embed - tu wykonuje sie zanurzenie
        o = e.narrow(1, 1, e.size(1) - 1)   # macierz e bez 1 kolumny (10x51xdim)
        s = e.narrow(1, 0, 1).expand_as(o)  # macierz 10x51xdim - wiersz to powt. sie 1 kolumna z e
        dist = self.manifold.distance(s, o)
        return dist.squeeze(-1), dist, e, o, s

    def loss(self, inp):
        return fun.cross_entropy(inp.neg(), target=zeros())
    
import pandas as pd
import random


def load_graph(vertices_num):
    edges_set = {i + 1: set() for i in range(vertices_num)}
    edges_pd = pd.read_csv('0_changed.edges', header=None, delimiter=" ")

    for i in range(len(edges_pd)):
        if edges_pd[0][i] <= vertices_num and edges_pd[1][i] <= vertices_num:
            edges_set[edges_pd[0][i]].add(edges_pd[1][i])
            edges_set[edges_pd[1][i]].add(edges_pd[0][i])

    # edges_set[2].add(3)
    # sprawdzam poprawnosc - uzywajac powyzszej linijki tworze blad w danych
    for num1 in edges_set:
        for num2 in edges_set:
            if num2 in edges_set[num1] and num1 not in edges_set[num2]:
                raise ValueError("Blad w danych")

    # usuwam zbiory puste - po zmianie pliku zakladam ze ich nie ma
    for j in list(edges_set):
        if edges_set[j] == set():
            print("Ucinam niepolaczony wierzch nr: ", j)
            edges_set.pop(j)

    return edges_set

P_Eukl = Euclidean()
graph = load_graph(330)
from random import choice, randint

model_n330 = Model(P_Eukl, 330, 2)
model_n10 = Model(P_Eukl, 10, 2)
model_n520 = Model(P_Eukl, 520, 2)
model_n331 = Model(P_Eukl, 331, 2) # model tworzy 331 wekt dwuwym, indeksowanych od 0 do 330

In [3]:
# zmieniam th.zeros na th.rand
inputs = th.randint(330, (10,52))
for i in range(0, 10): 
    batch = i+1
    batch_connected = choice(tuple(graph[batch]))
    inputs[i, 0] = batch
    inputs[i, 1] = batch_connected
    for j in range(2, 52):  
        inputs[i, j] = randint(1, 330)
        while inputs[i, j].item() in graph[batch]:
            inputs[i, j] = randint(1, 330)
            
# model_n331(inputs)            

In [15]:
from torch.optim.optimizer import Optimizer
optimizer = th.optim.SGD(model_n331.parameters(), lr=0.1)


In [3]:
import torch as th
th.randint(5, (20,20))

tensor([[2, 2, 3, 3, 4, 0, 0, 3, 4, 3, 0, 0, 3, 1, 2, 3, 2, 3, 2, 4],
        [0, 0, 0, 2, 3, 3, 1, 0, 4, 3, 2, 3, 2, 1, 3, 1, 3, 2, 3, 0],
        [4, 3, 1, 0, 4, 4, 0, 0, 1, 4, 3, 3, 0, 0, 3, 3, 1, 3, 2, 4],
        [4, 2, 0, 3, 1, 4, 1, 3, 3, 2, 2, 3, 3, 2, 4, 1, 4, 2, 1, 3],
        [1, 1, 4, 4, 3, 4, 0, 3, 1, 2, 1, 1, 1, 3, 1, 1, 4, 3, 4, 4],
        [1, 2, 3, 4, 3, 2, 1, 4, 3, 3, 4, 0, 2, 4, 3, 4, 1, 4, 0, 2],
        [4, 4, 2, 2, 1, 1, 4, 0, 4, 2, 2, 3, 3, 4, 1, 4, 1, 3, 3, 4],
        [2, 3, 4, 1, 3, 4, 1, 3, 2, 2, 0, 0, 4, 2, 4, 2, 0, 4, 3, 4],
        [4, 1, 2, 0, 2, 1, 0, 0, 0, 4, 0, 0, 4, 0, 3, 1, 3, 0, 1, 0],
        [0, 0, 4, 0, 4, 2, 0, 4, 1, 3, 4, 1, 3, 4, 2, 0, 0, 2, 2, 3],
        [1, 1, 1, 2, 0, 1, 4, 2, 4, 0, 1, 3, 3, 0, 2, 3, 2, 4, 0, 4],
        [2, 1, 2, 1, 1, 1, 2, 4, 2, 1, 2, 2, 4, 1, 2, 0, 1, 0, 1, 3],
        [3, 1, 0, 2, 4, 4, 3, 3, 1, 0, 4, 3, 3, 0, 0, 4, 0, 3, 0, 2],
        [3, 1, 0, 3, 2, 2, 4, 1, 2, 1, 0, 2, 3, 1, 3, 2, 4, 1, 2, 4],
        [0, 1, 1, 0,

[20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20]